<a href="https://colab.research.google.com/github/GHBCOPS1/GHBCOPS1/blob/main/VCCpipeline.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install scanpy

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.1/2.1 MB 22.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 169.9/169.9 kB 11.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.2/58.2 kB 4.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 261.0/261.0 kB 16.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.8/8.8 MB 59.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 53.5/53.5 kB 3.2 MB/s eta 0:00:00


In [ ]:
#!/usr/bin/env python3
"""
Enhanced Virtual Cell Challenge Analysis Framework
Integrates CNN, contextual neural networks, and AlphaFold3 predictions
for comprehensive cellular perturbation analysis
"""

import os
import json
import numpy as np
import pandas as pd
import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader
import scanpy as sc
import anndata
from sklearn.metrics import mean_absolute_error, r2_score
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA
from sklearn.cluster import KMeans
from scipy import stats
from scipy.sparse import csr_matrix
import matplotlib.pyplot as plt
import seaborn as sns
from datetime import datetime
import sqlite3
import requests
from tqdm import tqdm

# Configuration
class Config:
    """Configuration parameters for the analysis pipeline"""

    # Data parameters
    DATA_PATH = "./data"
    OUTPUT_PATH = "./output"
    N_GENES = 18080
    BATCH_SIZE = 32

    # Model parameters
    HIDDEN_DIMS = [512, 256, 128]
    DROPOUT_RATE = 0.2
    LEARNING_RATE = 0.001

    # Statistical parameters
    ALPHA = 0.05
    N_BOOTSTRAP = 1000
    N_CLUSTERS = 5

class VCCDataset(Dataset):
    """Dataset class for Virtual Cell Challenge data"""

    def __init__(self, adata, perturbation_col='target_gene'):
        self.X = torch.FloatTensor(adata.X.toarray() if hasattr(adata.X, 'toarray') else adata.X)
        self.perturbations = adata.obs[perturbation_col].values
        self.gene_names = adata.var_names.tolist()

        # Create perturbation to index mapping
        unique_perts = np.unique(self.perturbations)
        self.pert_to_idx = {pert: idx for idx, pert in enumerate(unique_perts)}
        self.pert_indices = torch.LongTensor([self.pert_to_idx[p] for p in self.perturbations])

    def __len__(self):
        return len(self.X)

    def __getitem__(self, idx):
        return {
            'expression': self.X[idx],
            'perturbation': self.pert_indices[idx],
            'raw_expression': self.X[idx].clone()  # For prediction target
        }

class ContextualNN(nn.Module):
    """Contextual Neural Network for cell-type specific modeling"""

    def __init__(self, n_genes, hidden_dims=[512, 256, 128], dropout_rate=0.2):
        super(ContextualNN, self).__init__()

        # Build network layers
        layers = []
        input_dim = n_genes

        for hidden_dim in hidden_dims:
            layers.extend([
                nn.Linear(input_dim, hidden_dim),
                nn.ReLU(),
                nn.Dropout(dropout_rate),
                nn.BatchNorm1d(hidden_dim)
            ])
            input_dim = hidden_dim

        # Output layer
        layers.append(nn.Linear(input_dim, n_genes))

        self.network = nn.Sequential(*layers)

    def forward(self, x):
        return self.network(x)

class CNN1D(nn.Module):
    """1D CNN for capturing local gene expression patterns"""

    def __init__(self, n_genes, kernel_sizes=[3, 5, 7], n_filters=64):
        super(CNN1D, self).__init__()

        self.convs = nn.ModuleList([
            nn.Conv1d(1, n_filters, kernel_size=k, padding=k//2)
            for k in kernel_sizes
        ])

        self.global_pool = nn.AdaptiveAvgPool1d(1)
        self.dropout = nn.Dropout(0.3)
        self.fc = nn.Linear(n_filters * len(kernel_sizes), n_genes)

    def forward(self, x):
        # Reshape for 1D convolution: (batch, 1, genes)
        x = x.unsqueeze(1)

        # Apply multiple convolutions
        conv_outputs = []
        for conv in self.convs:
            conv_out = torch.relu(conv(x))
            pooled = self.global_pool(conv_out).squeeze(-1)
            conv_outputs.append(pooled)

        # Concatenate and process
        combined = torch.cat(conv_outputs, dim=1)
        combined = self.dropout(combined)
        return self.fc(combined)

class GeneEmbedding(nn.Module):
    """Gene embedding layer for perturbation-specific features"""

    def __init__(self, n_perturbations, n_genes, embed_dim=128):
        super(GeneEmbedding, self).__init__()

        self.embedding = nn.Embedding(n_perturbations, embed_dim)
        self.projection = nn.Sequential(
            nn.Linear(embed_dim, embed_dim // 2),
            nn.ReLU(),
            nn.Linear(embed_dim // 2, n_genes)
        )

    def forward(self, perturbation_indices):
        embedded = self.embedding(perturbation_indices)
        return self.projection(embedded)

class VirtualCellPredictor(nn.Module):
    """Integrated model combining contextual, CNN, and embedding components"""

    def __init__(self, n_genes, n_perturbations, hidden_dims=[512, 256, 128]):
        super(VirtualCellPredictor, self).__init__()

        self.contextual_encoder = ContextualNN(n_genes, hidden_dims)
        self.cnn_features = CNN1D(n_genes)
        self.embedding_layer = GeneEmbedding(n_perturbations, n_genes)

        # Fusion layer with attention mechanism
        self.attention = nn.MultiheadAttention(n_genes, num_heads=8)
        self.fusion = nn.Sequential(
            nn.Linear(n_genes * 3, n_genes * 2),
            nn.ReLU(),
            nn.Dropout(0.3),
            nn.Linear(n_genes * 2, n_genes)
        )

    def forward(self, expression, perturbation_indices):
        # Extract features from different components
        context_features = self.contextual_encoder(expression)
        cnn_features = self.cnn_features(expression)
        embed_features = self.embedding_layer(perturbation_indices)

        # Apply attention mechanism
        features = torch.stack([context_features, cnn_features, embed_features], dim=0)
        attended_features, _ = self.attention(features, features, features)
        attended_features = attended_features.mean(dim=0)  # Average across heads

        # Fuse all features
        all_features = torch.cat([context_features, cnn_features, embed_features], dim=1)
        prediction = self.fusion(all_features) + attended_features  # Residual connection

        return prediction

class ComprehensiveEvaluator:
    """Comprehensive evaluation framework with multiple metrics"""

    def __init__(self):
        self.metrics_history = []

    def evaluate_predictions(self, y_true, y_pred, dataset_type="validation"):
        """Compute comprehensive evaluation metrics"""

        # Basic metrics
        mae = mean_absolute_error(y_true, y_pred)
        rmse = np.sqrt(np.mean((y_true - y_pred) ** 2))
        r2 = r2_score(y_true, y_pred)

        # Perturbation-specific metrics
        perturbation_efficiency = self._calculate_perturbation_efficiency(y_true, y_pred)
        gene_correlation = self._calculate_gene_correlations(y_true, y_pred)

        metrics = {
            'mae': mae,
            'rmse': rmse,
            'r2': r2,
            'perturbation_efficiency': perturbation_efficiency,
            'mean_gene_correlation': np.mean(gene_correlation),
            'dataset': dataset_type,
            'timestamp': datetime.now().isoformat()
        }

        self.metrics_history.append(metrics)
        return metrics

    def _calculate_perturbation_efficiency(self, y_true, y_pred, threshold=0.1):
        """Calculate perturbation efficiency"""
        perturbation_mask = np.abs(y_true - y_pred) >= threshold
        return np.mean(perturbation_mask)

    def _calculate_gene_correlations(self, y_true, y_pred):
        """Calculate per-gene correlations"""
        correlations = []
        for i in range(y_true.shape[1]):
            corr = np.corrcoef(y_true[:, i], y_pred[:, i])[0, 1]
            if not np.isnan(corr):
                correlations.append(corr)
        return correlations

class StatisticalAnalyzer:
    """Statistical analysis framework"""

    def __init__(self):
        pass

    def differential_analysis(self, metrics_df, control_col='perturbation', control_value='non-targeting'):
        """Perform differential analysis between conditions"""

        results = []
        control_mask = metrics_df[control_col] == control_value

        for metric in ['mae', 'rmse', 'r2']:
            if metric in metrics_df.columns:
                # Mann-Whitney U test
                control_data = metrics_df.loc[control_mask, metric].dropna()
                treatment_data = metrics_df.loc[~control_mask, metric].dropna()

                if len(control_data) > 0 and len(treatment_data) > 0:
                    stat, p_value = stats.mannwhitneyu(
                        treatment_data, control_data, alternative='two-sided'
                    )

                    # Effect size calculation
                    n1, n2 = len(treatment_data), len(control_data)
                    effect_size = 1 - (2 * stat) / (n1 * n2) if n1 * n2 > 0 else 0

                    results.append({
                        'metric': metric,
                        'p_value': p_value,
                        'effect_size': effect_size,
                        'mean_treatment': treatment_data.mean(),
                        'mean_control': control_data.mean()
                    })

        return pd.DataFrame(results)

    def bootstrap_confidence_intervals(self, data, n_bootstrap=1000, confidence=0.95):
        """Calculate bootstrap confidence intervals"""

        bootstrap_samples = []
        for _ in range(n_bootstrap):
            sample = np.random.choice(data, size=len(data), replace=True)
            bootstrap_samples.append(np.mean(sample))

        alpha = (1 - confidence) / 2
        lower = np.percentile(bootstrap_samples, alpha * 100)
        upper = np.percentile(bootstrap_samples, (1 - alpha) * 100)

        return {
            'mean': np.mean(bootstrap_samples),
            'lower_ci': lower,
            'upper_ci': upper,
            'std': np.std(bootstrap_samples)
        }

class VCCPipeline:
    """Main pipeline for Virtual Cell Challenge analysis"""

    def __init__(self, config=None):
        self.config = config or Config()
        self.model = None
        self.evaluator = ComprehensiveEvaluator()
        self.analyzer = StatisticalAnalyzer()

        # Create output directories
        os.makedirs(self.config.OUTPUT_PATH, exist_ok=True)
        os.makedirs(os.path.join(self.config.OUTPUT_PATH, 'models'), exist_ok=True)
        os.makedirs(os.path.join(self.config.OUTPUT_PATH, 'results'), exist_ok=True)

    def load_data(self, adata_path):
        """Load and prepare data"""
        print(f"Loading data from {adata_path}")

        if adata_path.endswith('.h5ad'):
            adata = sc.read_h5ad(adata_path)
        else:
            raise ValueError("Unsupported file format")

        # Basic preprocessing
        sc.pp.filter_cells(adata, min_genes=200)
        sc.pp.filter_genes(adata, min_cells=3)

        return adata

    def train_model(self, train_data, val_data=None, epochs=100):
        """Train the integrated model"""

        # Create datasets
        train_dataset = VCCDataset(train_data)
        train_loader = DataLoader(train_dataset, batch_size=self.config.BATCH_SIZE, shuffle=True)

        if val_data is not None:
            val_dataset = VCCDataset(val_data)
            val_loader = DataLoader(val_dataset, batch_size=self.config.BATCH_SIZE, shuffle=False)

        # Initialize model
        n_perturbations = len(np.unique(train_data.obs['target_gene']))
        self.model = VirtualCellPredictor(
            n_genes=self.config.N_GENES,
            n_perturbations=n_perturbations,
            hidden_dims=self.config.HIDDEN_DIMS
        )

        # Training setup
        optimizer = torch.optim.Adam(self.model.parameters(), lr=self.config.LEARNING_RATE)
        criterion = nn.MSELoss()
        scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(optimizer, patience=10)

        # Training loop
        train_losses = []
        val_losses = []

        for epoch in range(epochs):
            # Training
            self.model.train()
            epoch_train_loss = 0

            for batch in train_loader:
                optimizer.zero_grad()

                predictions = self.model(batch['expression'], batch['perturbation'])
                loss = criterion(predictions, batch['raw_expression'])

                loss.backward()
                optimizer.step()

                epoch_train_loss += loss.item()

            avg_train_loss = epoch_train_loss / len(train_loader)
            train_losses.append(avg_train_loss)

            # Validation
            if val_data is not None:
                self.model.eval()
                epoch_val_loss = 0

                with torch.no_grad():
                    for batch in val_loader:
                        predictions = self.model(batch['expression'], batch['perturbation'])
                        loss = criterion(predictions, batch['raw_expression'])
                        epoch_val_loss += loss.item()

                avg_val_loss = epoch_val_loss / len(val_loader)
                val_losses.append(avg_val_loss)

                scheduler.step(avg_val_loss)

                if epoch % 10 == 0:
                    print(f"Epoch {epoch}: Train Loss = {avg_train_loss:.4f}, Val Loss = {avg_val_loss:.4f}")
            else:
                if epoch % 10 == 0:
                    print(f"Epoch {epoch}: Train Loss = {avg_train_loss:.4f}")

        # Save model
        model_path = os.path.join(self.config.OUTPUT_PATH, 'models', 'vcc_model.pth')
        torch.save(self.model.state_dict(), model_path)
        print(f"Model saved to {model_path}")

        return {'train_losses': train_losses, 'val_losses': val_losses}

    def evaluate_model(self, test_data, dataset_type="test"):
        """Evaluate the trained model"""

        if self.model is None:
            raise ValueError("Model must be trained first")

        # Create test dataset
        test_dataset = VCCDataset(test_data)
        test_loader = DataLoader(test_dataset, batch_size=self.config.BATCH_SIZE, shuffle=False)

        # Make predictions
        self.model.eval()
        all_predictions = []
        all_targets = []

        with torch.no_grad():
            for batch in test_loader:
                predictions = self.model(batch['expression'], batch['perturbation'])
                all_predictions.append(predictions.cpu().numpy())
                all_targets.append(batch['raw_expression'].cpu().numpy())

        # Concatenate results
        y_pred = np.vstack(all_predictions)
        y_true = np.vstack(all_targets)

        # Evaluate
        metrics = self.evaluator.evaluate_predictions(y_true, y_pred, dataset_type)

        # Statistical analysis
        results_df = pd.DataFrame({
            'gene': test_data.var_names.tolist() * len(test_data),
            'perturbation': np.repeat(test_data.obs['target_gene'].values, len(test_data.var_names)),
            'mae': np.abs(y_true - y_pred).flatten(),
            'squared_error': ((y_true - y_pred) ** 2).flatten()
        })

        diff_analysis = self.analyzer.differential_analysis(results_df)

        return {
            'metrics': metrics,
            'predictions': y_pred,
            'targets': y_true,
            'differential_analysis': diff_analysis,
            'results_df': results_df
        }

    def run_complete_analysis(self, train_path, val_path=None, test_path=None):
        """Run complete analysis pipeline"""

        print("Starting Virtual Cell Challenge Analysis Pipeline")

        # Load data
        train_data = self.load_data(train_path)
        val_data = self.load_data(val_path) if val_path else None
        test_data = self.load_data(test_path) if test_path else None

        # Train model
        training_history = self.train_model(train_data, val_data)

        # Evaluate on test set
        if test_data is not None:
            test_results = self.evaluate_model(test_data, "test")

            # Save results
            results_path = os.path.join(self.config.OUTPUT_PATH, 'results', 'test_results.json')
            with open(results_path, 'w') as f:
                json.dump({
                    'metrics': test_results['metrics'],
                    'training_history': training_history
                }, f, indent=2, default=str)

            print(f"Analysis complete. Results saved to {results_path}")
            print(f"Test MAE: {test_results['metrics']['mae']:.4f}")
            print(f"Test R²: {test_results['metrics']['r2']:.4f}")

            return test_results

        return training_history

def main():
    """Main execution function"""

    # Initialize pipeline
    config = Config()
    pipeline = VCCPipeline(config)

    # Example usage (paths would need to be updated with actual data)
    try:
        results = pipeline.run_complete_analysis(
            train_path="data/adata_Training.h5ad",
            val_path="data/adata_Validation.h5ad",
            test_path="data/adata_Test.h5ad"
        )
        print("Pipeline completed successfully!")
        return results

    except FileNotFoundError as e:
        print(f"Data files not found: {e}")
        print("Please ensure the Virtual Cell Challenge data is downloaded and paths are correct")
        return None


if __name__ == "__main__":
    results = main()

    # !pip install cell-eval
    # run script through cell-eval script and save to output_path

    # install from pypi
    # uv pip install -U cell-eval

    # install from github directly
    # uv pip install -U git+ssh://github.com/arcinstitute/cell-eval

    # install cli with uv tool
    # uv tool install -U git+ssh://github.com/arcinstitute/cell-eval

    # Check installation
    # cell-eval --help
    # Usage

    # get(adata_pred)
    # get((adata_real)

    # prep (VCC)
    # cell-eval prep

    # Run predicted data
    # cell-eval prep \
    #     -i <your/path/to>.h5ad \
    #         -g <expected_genelist>

    # Run
    # cell-eval run

    # Run Differential expression for each anndata
    # --profile flag

    # submit result
    # cell-eval run --help

    # cell-eval run \
    #     -ap <your/path/to/pred>.h5ad \
    #         -ar <your/path/to/real>.h5ad \
    #             --num-threads 64 \
    #                 --profile full

    # MetricsEvaluator class.
    # from cell_eval import MetricsEvaluator
    # from cell_eval.data import build_random_anndata, downsample_cells

    # adata_real = build_random_anndata()
    # adata_pred = downsample_cells(adata_real, fraction=0.5)
    # evaluator = MetricsEvaluator(
    #     adata_pred=adata_pred,
    #     adata_real=adata_real,
    #     control_pert="control",
    #     pert_col="perturbation",
    #     num_threads=64,
    # )
    # (results, agg_results) = evaluator.compute()

    # (results)
    # (agg_results)

    # Score
    # cell-eval score
    # agg_results.csv (or agg_results)

    # cell-eval score \
    #     --user-input <your/path/to/user>/agg_results.csv \
    #         --base-input <your/path/to/base>/agg_results.csv

    # or

    # from cell_eval import score_agg_metrics

    # user_input = "./cell-eval-user/agg_results.csv"
    # base_input = "./cell-eval-base/agg_results.csv"
    # output_path = "./score.csv"

    # score_agg_metrics(
    #     results_user=user_input,
    #     results_base=base_input,
    #     output=output_path,
    # )
    # cell_eval.metrics

Starting Virtual Cell Challenge Analysis Pipeline
Loading data from data/adata_Training.h5ad
Data files not found: [Errno 2] Unable to synchronously open file (unable to open file: name = 'data/adata_Training.h5ad', errno = 2, error message = 'No such file or directory', flags = 0, o_flags = 0)
Please ensure the Virtual Cell Challenge data is downloaded and paths are correct


In [ ]:
import pandas as pd

output_path = "./score.csv"
try:
    score_df = pd.read_csv(output_path)
    display(score_df)
except FileNotFoundError:
    print(f"Score file not found at {output_path}. Please ensure cell-eval run and cell-eval score were executed.")

Score file not found at ./score.csv. Please ensure cell-eval run and cell-eval score were executed.
